In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [4]:
# split the training set into 60% and 40%, so we'll end up with 15000 examples for trainig, 10000 examples
# for validation and 25000 examples for test

train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
                name="imdb_reviews",
                split=(train_validation_split, tfds.Split.TEST),
                as_supervised=True)

In [5]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_example_batch

<tf.Tensor: id=439, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [6]:
train_labels_batch

<tf.Tensor: id=440, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)>

In [7]:
embedding = "https://hub.tensorflow.google.cn/google/tf2-preview/gnews-swivel-20dim/1"

In [8]:
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                          dtype=tf.string, trainable=True)
hub_layer(train_example_batch[:3])

<tf.Tensor: id=620, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [9]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [10]:
# loss: binary classification --> binary_crossentropy
# binary_crossentropy is better for dealing with probabilities
# regression problems  --> mean squared error.

In [11]:
model.compile(optimizer="adam",
             loss="binary_crossentropy",
             metrics=["accuracy"])

In [12]:
history = model.fit(train_data.shuffle(10000).batch(512),
                   epochs=20,
                   validation_data=validation_data.batch(512),
                   verbose=1)

Epoch 1/20


W1114 14:22:04.280811 102032 deprecation.py:323] From C:\Users\dyb\Anaconda3\lib\site-packages\tensorflow_core\python\ops\nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================].2603 - accuracy: 0.50 - 2s 1s/step - loss: 1.2669 - accuracy: 0.49 - 2s 750ms/step - loss: 1.2418 - accuracy: 0.503 - 2s 591ms/step - loss: 1.2270 - accuracy: 0.497 - 2s 493ms/step - loss: 1.2155 - accuracy: 0.489 - 3s 430ms/step - loss: 1.2016 - accuracy: 0.485 - 3s 383ms/step - loss: 1.1679 - accuracy: 0.490 - 3s 348ms/step - loss: 1.1411 - accuracy: 0.493 - 3s 322ms/step - loss: 1.1252 - accuracy: 0.490 - 3s 299ms/step - loss: 1.1115 - accuracy: 0.489 - 3s 281ms/step - loss: 1.0913 - accuracy: 0.489 - 3s 266ms/step - loss: 1.0743 - accuracy: 0.489 - 3s 252ms/step - loss: 1.0591 - accuracy: 0.488 - 3s 240ms/step - loss: 1.0438 - accuracy: 0.485 - 3s 230ms/step - loss: 1.0269 - accuracy: 0.485 - 4s 221ms/step - loss: 1.0111 - accuracy: 0.485 - 4s 213ms/step - loss: 0.9987 - accuracy: 0.482 - 4s 207ms/step - loss: 0.9874 - accuracy: 0.480 - 4s 201ms/step - loss: 0.9760 - accuracy: 0.478 - 4s 196ms/step - loss: 0.9666 - accuracy: 0.474 - 4s 192ms/s

30/30 [==============================] - ETA: 15s - loss: 0.4072 - accuracy: 0.826 - ETA: 8s - loss: 0.4147 - accuracy: 0.824 - ETA: 6s - loss: 0.4169 - accuracy: 0.82 - ETA: 4s - loss: 0.4151 - accuracy: 0.82 - ETA: 3s - loss: 0.4146 - accuracy: 0.82 - ETA: 3s - loss: 0.4159 - accuracy: 0.82 - ETA: 2s - loss: 0.4135 - accuracy: 0.82 - ETA: 2s - loss: 0.4132 - accuracy: 0.82 - ETA: 2s - loss: 0.4116 - accuracy: 0.82 - ETA: 1s - loss: 0.4105 - accuracy: 0.82 - ETA: 1s - loss: 0.4108 - accuracy: 0.82 - ETA: 1s - loss: 0.4116 - accuracy: 0.82 - ETA: 1s - loss: 0.4105 - accuracy: 0.82 - ETA: 0s - loss: 0.4092 - accuracy: 0.82 - ETA: 0s - loss: 0.4095 - accuracy: 0.82 - ETA: 0s - loss: 0.4086 - accuracy: 0.83 - ETA: 0s - loss: 0.4090 - accuracy: 0.82 - ETA: 0s - loss: 0.4076 - accuracy: 0.82 - ETA: 0s - loss: 0.4073 - accuracy: 0.82 - 6s 185ms/step - loss: 0.4040 - accuracy: 0.8349 - val_loss: 0.4115 - val_accuracy: 0.8215
Epoch 10/20
30/30 [==============================] - ETA: 14s - loss

30/30 [==============================] - ETA: 16s - loss: 0.2001 - accuracy: 0.935 - ETA: 9s - loss: 0.1841 - accuracy: 0.940 - ETA: 6s - loss: 0.1971 - accuracy: 0.92 - ETA: 5s - loss: 0.1987 - accuracy: 0.92 - ETA: 4s - loss: 0.2024 - accuracy: 0.92 - ETA: 3s - loss: 0.1932 - accuracy: 0.93 - ETA: 3s - loss: 0.1966 - accuracy: 0.93 - ETA: 2s - loss: 0.1971 - accuracy: 0.93 - ETA: 2s - loss: 0.1981 - accuracy: 0.93 - ETA: 2s - loss: 0.1981 - accuracy: 0.93 - ETA: 1s - loss: 0.1974 - accuracy: 0.93 - ETA: 1s - loss: 0.1966 - accuracy: 0.93 - ETA: 1s - loss: 0.1954 - accuracy: 0.93 - ETA: 1s - loss: 0.1967 - accuracy: 0.93 - ETA: 0s - loss: 0.1974 - accuracy: 0.93 - ETA: 0s - loss: 0.1979 - accuracy: 0.93 - ETA: 0s - loss: 0.1976 - accuracy: 0.93 - ETA: 0s - loss: 0.1980 - accuracy: 0.93 - ETA: 0s - loss: 0.1975 - accuracy: 0.93 - 6s 192ms/step - loss: 0.1983 - accuracy: 0.9299 - val_loss: 0.3022 - val_accuracy: 0.8735
Epoch 18/20
30/30 [==============================] - ETA: 16s - loss

In [13]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 4s - loss: 0.3178 - accuracy: 0.8667
loss: 0.318
accuracy: 0.867
